In [ ]:
!python -m venv venv
!venv\Scripts\activate

In [ ]:
!venv\Scripts\activate

In [3]:
import os
import pandas as pd

In [30]:
data_folder = './'
unique_narrator_ids = set()

In [31]:
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        df = df.dropna(subset=['Hadith_number'])
        narrators_list = df['Narrators'].str.split(',').explode()
        unique_narrator_ids.update(narrators_list.dropna().str.strip())
        df.to_csv(file_path, index=False)

        print(f"Processed {filename}")

Processed sunanIbnMajah.csv
Processed sahihMuslim.csv
Processed sunanAnNasai.csv
Processed sunanAbiDaud.csv
Processed sahihBukhari.csv
Processed jamiAlTirmidhi.csv


In [32]:
print(len(unique_narrator_ids))

4052


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def fetch_narrator_data(id):
    base_url = "https://muslimscholars.info/manage.php?manage=&scholarSearch=&submit=scholar&yfield=&ID="
    url = base_url + str(id)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'align': 'CENTER', 'class': 'form', 'width': '100%'})

        if table:
            full_name = None
            birth_date = None
            birth_place = None
            death_date = None
            death_place = None
            places_of_stay = None
            scholar_name = None

            rows = table.find_all('tr')

            for row in rows:
                scholar_font = soup.find('font', {'face': 'naskh, Verdana, Arial, Helvetica, sans-serif'})
                scholar_name = scholar_font.get_text(strip=True) if scholar_font else None

                if "Full Name:" in row.text:
                    full_name = row.find('td', {'colspan': '2'}).text.strip()

                if "Birth Date/Place:" in row.text:
                    birth_data = row.find('td', {'colspan': '2'}).text.strip()
                    birth_match = re.match(r"(.+)\s\((.+)\)", birth_data)
                    if birth_match:
                        birth_date = birth_match.group(1).strip()
                        birth_place = birth_match.group(2).strip()

                if "Death Date/Place:" in row.text:
                    death_data = row.find('td', {'colspan': '2'}).text.strip()
                    death_match = re.match(r"(.+)\s\((.+)\)", death_data)
                    if death_match:
                        death_date = death_match.group(1).strip()
                        death_place = death_match.group(2).strip()

                if "Places of Stay:" in row.text:
                    places_of_stay = row.find('td', {'colspan': '2'}).text.strip()

            english_part = re.search(r"[A-Za-z0-9'\- ]+", scholar_name)
            english_text = english_part.group(0).strip() if english_part else None

            arabic_text = scholar_name.replace(english_text, "").strip() if english_text else None

            return {
                'ID': id,
                'Full Name': full_name,
                'Birth Date': birth_date,
                'Birth Place': birth_place,
                'Death Date': death_date,
                'Death Place': death_place,
                'Places of Stay': places_of_stay,
                "Name": english_text,
                "Arabic Name": arabic_text
            }
        else:
            print(f"Table not found for ID: {id}")
            return None
    else:
        print(f"Failed to retrieve the webpage for ID: {id}. Status code: {response.status_code}")
        return None


results = []

for id in unique_narrator_ids:
    result = fetch_narrator_data(id)
    if result:
        results.append(result)

df = pd.DataFrame(results)
df.to_csv('narrators_data.csv', index=False)
print("Data saved to narrators_data.csv")

Data saved to narrators_data.csv


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def fetch_narrator_data(id):
    base_url = "https://muslimscholars.info/manage.php?manage=&scholarSearch=&submit=scholar&yfield=&ID="
    url = base_url + str(id)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'align': 'CENTER', 'class': 'form', 'width': '100%'})

        if table:
            full_name = None
            birth_date = None
            birth_place = None
            death_date = None
            death_place = None
            places_of_stay = None
            scholar_name = None

            rows = table.find_all('tr')

            for row in rows:
                scholar_font = soup.find('font', {'face': 'naskh, Verdana, Arial, Helvetica, sans-serif'})
                scholar_name = scholar_font.get_text(strip=True) if scholar_font else None


                if "Full Name:" in row.text:
                    full_name = row.find('td', {'colspan': '2'}).text.strip()

                if "Birth Date/Place:" in row.text:
                    birth_data = row.find('td', {'colspan': '2'}).text.strip()
                    birth_match = re.match(r"(.+)\s\((.+)\)", birth_data)
                    if birth_match:
                        birth_date = birth_match.group(1).strip()
                        birth_place = birth_match.group(2).strip()

                if "Death Date/Place:" in row.text:
                    death_data = row.find('td', {'colspan': '2'}).text.strip()
                    death_match = re.match(r"(.+)\s\((.+)\)", death_data)
                    if death_match:
                        death_date = death_match.group(1).strip()
                        death_place = death_match.group(2).strip()

                if "Places of Stay:" in row.text:
                    places_of_stay = row.find('td', {'colspan': '2'}).text.strip()

            english_part = re.search(r"[A-Za-z0-9'\- ]+", scholar_name)
            english_text = english_part.group(0).strip() if english_part else None


            arabic_text = scholar_name.replace(english_text, "").strip() if english_text else None

            return {
                'ID': id,
                'Full Name': full_name,
                'Birth Date': birth_date,
                'Birth Place': birth_place,
                'Death Date': death_date,
                'Death Place': death_place,
                'Places of Stay': places_of_stay,
                "Name":english_text,
                "arabic Name":arabic_text
            }
        else:
            print(f"Table not found for ID: {id}")
            return None
    else:
        print(f"Failed to retrieve the webpage for ID: {id}. Status code: {response.status_code}")
        return None

# Example usage for a single ID
narrator_id = 20469
result = fetch_narrator_data(narrator_id)
print(result)


{'ID': 20469, 'Full Name': "Wadah bin 'Abdullah al-Yashkari", 'Birth Date': '122', 'Birth Place': 'Basra', 'Death Date': None, 'Death Place': None, 'Places of Stay': 'Basra,al-Wasit', 'Name': "Wadah bin 'Abdullah al-Yashkari", 'arabic Name': 'وضاح بن عبد الله اليشكري  أبو عوانة'}
